# Storing Data and Running Queries

Putting everything into a database so we can query it with SQL.

In [1]:
import os
import sys

# Windows Hadoop fix
os.environ['HADOOP_HOME'] = r'C:\hadoop'
os.environ['hadoop.home.dir'] = r'C:\hadoop'
os.environ['PATH'] = r'C:\hadoop\bin;' + os.environ['PATH']

print("Hadoop configured for Windows")

Hadoop configured for Windows


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder.appName("HiveStorage").config("spark.driver.memory", "4g")
        .config("spark.sql.warehouse.dir", "spark-warehouse").enableHiveSupport().getOrCreate()

enriched = spark.read.parquet("output/enriched_data.parquet")

In [3]:
spark.sql("CREATE DATABASE IF NOT EXISTS airbnb_analytics")
spark.sql("USE airbnb_analytics")

DataFrame[]

Creating main table:

In [4]:
enriched.write.mode("overwrite").partitionBy("price_band").saveAsTable("listings_enriched")
print("Main table created")

Main table created


Creating summary tables:

In [5]:
neighbourhood_stats = enriched.groupBy("neighbourhood").agg(
    count("*").alias("listing_count"),
    avg("price").alias("avg_price"),
    avg("review_scores_rating").alias("avg_rating"),
    avg("availability_rate").alias("avg_availability")
)
neighbourhood_stats.write.mode("overwrite").saveAsTable("neighbourhood_stats")

room_type_dist = enriched.groupBy("room_type").agg(
    count("*").alias("count"),
    avg("price").alias("avg_price")
)
room_type_dist.write.mode("overwrite").saveAsTable("room_type_distribution")

price_analysis = enriched.groupBy("price_band").agg(
    count("*").alias("count"),
    avg("number_of_reviews").alias("avg_reviews"),
    avg("review_scores_rating").alias("avg_rating")
)
price_analysis.write.mode("overwrite").saveAsTable("price_analysis")

host_performance = enriched.groupBy("host_id").agg(
    count("*").alias("total_listings"),
    avg("price").alias("avg_price"),
    avg("review_scores_rating").alias("avg_rating"),
    sum("number_of_reviews").alias("total_reviews")
)
host_performance.write.mode("overwrite").saveAsTable("host_performance")

print("All tables created")

All tables created


In [6]:
spark.sql("SHOW TABLES").show()

+----------------+--------------------+-----------+
|       namespace|           tableName|isTemporary|
+----------------+--------------------+-----------+
|airbnb_analytics|    host_performance|      false|
|airbnb_analytics|   listings_enriched|      false|
|airbnb_analytics| neighbourhood_stats|      false|
|airbnb_analytics|      price_analysis|      false|
|airbnb_analytics|room_type_distrib...|      false|
+----------------+--------------------+-----------+



Running some example queries:

In [7]:
print("Average price by price band:")
spark.sql("""
    SELECT price_band, AVG(price) as avg_price, COUNT(*) as count
    FROM listings_enriched
    GROUP BY price_band
    ORDER BY avg_price DESC
""").show()

Average price by price band:
+----------+------------------+-----+
|price_band|         avg_price|count|
+----------+------------------+-----+
|      high| 657.6716451181453| 8972|
|    medium|173.67488240221664|31038|
|       low|63.578700328107914|21944|
+----------+------------------+-----+



In [8]:
print("Most expensive neighbourhoods:")
spark.sql("""
    SELECT neighbourhood, avg_price, listing_count
    FROM neighbourhood_stats
    ORDER BY avg_price DESC
    LIMIT 10
""").show()

Most expensive neighbourhoods:
+--------------------+------------------+-------------+
|       neighbourhood|         avg_price|listing_count|
+--------------------+------------------+-------------+
|Neighborhood high...| 220.9838064937184|        24516|
|             Unknown|194.15254554196272|        37438|
+--------------------+------------------+-------------+



In [9]:
print("Room types:")
spark.sql("""
    SELECT room_type, count, avg_price
    FROM room_type_distribution
    ORDER BY count DESC
""").show()

Room types:
+---------------+-----+------------------+
|      room_type|count|         avg_price|
+---------------+-----+------------------+
|Entire home/apt|42316|251.96535589375176|
|   Private room|19375|101.07261935483871|
|    Shared room|  191| 96.91099476439791|
|     Hotel room|   72| 657.8333333333334|
+---------------+-----+------------------+

